In [ ]:
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers

import tensorflow as tf

from keras.preprocessing.image import array_to_img, img_to_array, load_img#,save_img

import time
t_start = time.time()

In [ ]:
# Set some parameters
im_width = 240
im_height = 240
border = 5
path_train = "C:/Users/yousu/Downloads/BlastsOnline"

# Get and resize train images and masks
def get_data(path, train=True):
    ids = next(os.walk(path + "/imagesResized"))[2]
    ids2= next(os.walk(path + "/NewLabels"))[2]
    X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    if train:
        y = np.zeros((len(ids2), im_height, im_width, 1), dtype=np.float32)
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        img = load_img(path + '/imagesResized/' + id_, grayscale=True)
        x_img = img_to_array(img)
        x_img = resize(x_img, (im_width, im_height, 1), mode='constant', preserve_range=True)
        X[n] = x_img / 255.0

    for n, id_ in tqdm_notebook(enumerate(ids2), total=len(ids2)):
        # Load masks
        if train:
            mask = img_to_array(load_img(path + '/NewLabels/' + id_, grayscale=True))
            mask = resize(mask, (im_width, im_height, 1), mode='constant', preserve_range=True)
            y[n] = mask / 255.0

        # Save images
        #X[n] = x_img / 255
        #if train:
            #y[n] = mask / 255
    print('Done!')
    if train:
        return X, y
    else:
        return X
    
X, y = get_data(path_train, train=True)

In [ ]:
# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2018)

In [ ]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.losses import bce_dice_loss
from segmentation_models.metrics import iou_score

In [ ]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3))
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [ ]:
# Build model
def build_model(input_layer, start_neurons, DropoutRatio = 0.5):
    # 101 -> 50
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(input_layer)
    #conv1 = residual_block(conv1,start_neurons * 1)
    conv1 = residual_block(conv1,start_neurons * 1, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(DropoutRatio/2)(pool1)

    # 50 -> 25
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(pool1)
    #conv2 = residual_block(conv2,start_neurons * 2)
    conv2 = residual_block(conv2,start_neurons * 2, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(DropoutRatio)(pool2)

    # 25 -> 12
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(pool2)
    #conv3 = residual_block(conv3,start_neurons * 4)
    conv3 = residual_block(conv3,start_neurons * 4, True)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(DropoutRatio)(pool3)

    # 12 -> 6
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(pool3)
    #conv4 = residual_block(conv4,start_neurons * 8)
    conv4 = residual_block(conv4,start_neurons * 8, True)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(DropoutRatio)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(pool4)
    #convm = residual_block(convm,start_neurons * 16)
    convm = residual_block(convm,start_neurons * 16, True)
    
    #Dilated Middle
    #convm = bottleneck(pool4, filters_bottleneck=start_neurons * 16, mode='cascade', depth=4,
               #kernel_size=(3, 3), activation='relu')
    #convm = residual_block(convm,start_neurons * 16, True)
    
    # 6 -> 12
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    #uconv4 = BatchNormalization()(uconv4)
    uconv4 = Dropout(DropoutRatio)(uconv4)
    
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv4)
    #uconv4 = residual_block(uconv4,start_neurons * 8)
    uconv4 = residual_block(uconv4,start_neurons * 8, True)
    
    # 12 -> 25
    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    #deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="valid")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    #uconv3 = BatchNormalization()(uconv3)
    uconv3 = Dropout(DropoutRatio)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv3)
    #uconv3 = residual_block(uconv3,start_neurons * 4)
    uconv3 = residual_block(uconv3,start_neurons * 4, True)

    # 25 -> 50
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    #uconv2 = BatchNormalization()(uconv2)
        
    uconv2 = Dropout(DropoutRatio)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv2)
    #uconv2 = residual_block(uconv2,start_neurons * 2)
    uconv2 = residual_block(uconv2,start_neurons * 2, True)
    
    # 50 -> 101
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    #deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="valid")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    #uconv1 = BatchNormalization()(uconv1)
    
    uconv1 = Dropout(DropoutRatio)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv1)
    #uconv1 = residual_block(uconv1,start_neurons * 1)
    uconv1 = residual_block(uconv1,start_neurons * 1, True)
    
    #uconv1 = Dropout(DropoutRatio/2)(uconv1)
    #output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    output_layer_noActi = Conv2D(1, (1,1), padding="same", activation=None)(uconv1)
    output_layer =  Activation('sigmoid')(output_layer_noActi)
    
    return output_layer

In [ ]:
# model
img_size_target = 240
input_layer = Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16,0.05)
lr=.0001
# del model
model_resunet1 = Model(input_layer, output_layer)
model_resunet1.compile(optimizer=Adam(lr), loss=bce_jaccard_loss, metrics=[iou_score])

model_resunet1.summary()

In [ ]:
# del model
model_resunet2 = Model(input_layer, output_layer)
model_resunet2.compile(optimizer=Adam(lr), loss=bce_dice_loss, metrics=[iou_score])

model_resunet2.summary()

In [ ]:
# del model
model_resunet3 = Model(input_layer, output_layer)
model_resunet3.compile(optimizer=Adam(lr), loss="binary_crossentropy", metrics=[iou_score])

model_resunet3.summary()

In [ ]:
# del model
model_resunet4 = Model(input_layer, output_layer)
model_resunet4.compile(optimizer=Adam(lr), loss="mean_squared_error", metrics=[iou_score])

model_resunet4.summary()

In [ ]:
model_resunet1.load_weights('model-ResUnet_aug.h5')
model_resunet2.load_weights('model-ResUnet_bcedice.h5')
model_resunet3.load_weights('model-ResUnet_bce.h5')
model_resunet4.load_weights('model-ResUnet_mse.h5')

In [ ]:
members = [model_resunet1,model_resunet2,model_resunet3,model_resunet4]

In [ ]:
def ensemble_predictions(members, weights, testX):
# make predictions
    yhats = [model.predict(testX) for model in members]
    yhats = np.array(yhats)
# sum across ensemble members
    avg = np.tensordot(yhats, weights, axes=((0),(0)))
# argmax across classes
    result = (avg > 0.5).astype(np.float32)  #Threshold 0.5
    return result

In [ ]:
# Equal weights
weights = [0.25,0.25,0.25,0.25]
#weights = [0, 0.33333333, 0, 0.66666667]
epred = ensemble_predictions(members,weights, X_valid)

In [ ]:
def plot_sample(X, y, preds, binary_preds, ix=None):
    if ix is None:
        ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='gray')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='b', levels=[0.5])
    ax[0].set_title('Embryo')

    ax[1].imshow(y[ix].squeeze(), vmin=0, vmax=2)
    ax[1].set_title('Embryo_GT')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Embryo Predicted')
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='r', levels=[1])
    ax[3].set_title('Embryo Predicted binary');

In [ ]:
# Check if valid data looks all right
plot_sample(X_valid, y_valid, epred, epred, ix=78)

In [ ]:
def jaccard(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, np.float32)
    y_pred = tf.convert_to_tensor(y_pred, np.float32)
    epsilon = 1e-15
    #intersection = (y_pred * y_true).sum(dim=-2).sum(dim=-1)
    #union = y_true.sum(dim=-2).sum(dim=-1) + y_pred.sum(dim=-2).sum(dim=-1)
    y_true_f = tf.layers.flatten(y_true)
    y_pred_f = tf.layers.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f)+tf.reduce_sum(y_pred_f)

    return (tf.reduce_mean(intersection + epsilon)/ (union - intersection + epsilon))

In [ ]:
def evaluate_ensemble(members, weights, testX, testy):
# make prediction
     yhat = ensemble_predictions(members, weights, testX)
# calculate accuracy
     with tf.Session() as sess:
          return sess.run(iou_score(testy, yhat))

In [ ]:
print(evaluate_ensemble(members,weights,X_valid,y_valid))

In [ ]:
from numpy.linalg import norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result

In [ ]:
from itertools import product
# grid search weights
#def grid_search(members):
    # define weights to consider
w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
best_weights = []
#c=0
# iterate all possible combinations (cartesian product)
for weights in product(w, repeat=4):
    # skip if all weights are equal
    #c=c+1
    if len(set(weights)) == 1:
        continue
    # hack, normalize weight vector
    weights = normalize(weights)
    #if any((weights == x).all() for x in best_weights) ==True:
        #continue
    # evaluate weights
    #score = evaluate_ensemble(members, weights, testX, testy)
    #if score > best_score:
    #best_weights = weights
        #print('>%s %.4f' % (best_weights, best_score))
    best_weights.append(weights)
    #print(c)
print(len(best_weights))

In [ ]:
def num_after_point(x):
    s = str(x)
    if not '.' in s:
        return 0
    return len(s) - s.index('.') - 1

In [ ]:
#best_weights[9]==best_weights[:9]
D=[]
for i in range (len(best_weights)):
    if any((best_weights[i-1] == x).all() for x in best_weights[:i-1]) ==False:
        if num_after_point(best_weights[i-1][0])<2:
            D.append(best_weights[i-1])
print(len(D))

In [ ]:
def grid_search(members, testX, testy):
    #x=0
    best_score, best_weights = 0.0, None
    for j in range (100):
        #best_score, best_weights = 0.0, None
        score = evaluate_ensemble(members, D[j-1], X_valid, y_valid)
        if score > best_score:
            best_score, best_weights = score, D[j-1]
            print('>%s %.5f' % (best_weights, best_score))
            #print(x)
    return list(best_weights)

In [ ]:
weights = grid_search(members,X_valid,y_valid)
score = evaluate_ensemble(members, weights, X_valid, y_valid)
print('Grid Search Weights: %s, Score: %.4f' % (weights, score))

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
#history = model.fit(Xtrain, ytrain, validation_split=0.3, epochs=10, verbose=0)

# evaluate the model
y_valid = tf.convert_to_tensor(y_valid, np.float32)
epred = tf.convert_to_tensor(epred, np.float32)
#preds_val = tf.convert_to_tensor(preds_val, np.float32)

In [ ]:
with tf.Session() as sess:
    print(sess.run(recall_m(y_valid,epred)))
    print(sess.run(precision_m(y_valid,epred)))
    print(sess.run(f1_m(y_valid,epred)))